In [1]:
import requests
import pandas as pd
import os
from dotenv import load_dotenv
import json

In [2]:
load_dotenv()
api_key = os.getenv('WATCHMODE_API_KEY')

In [ ]:
""" Find and grab the Horror genre id. """
url = 'https://api.watchmode.com/v1/genres'
params = {'apiKey': api_key}
response = requests.get(url, params=params)
genres = response.json()
genres

In [31]:
# Grab the first page of horror movies.
url = 'https://api.watchmode.com/v1/list-titles'
params = {
    'apiKey': api_key,
    'genres': 11,
    'types': 'movie',
    'page': 1
}

response = requests.get(url, params=params)
data = response.json()

# Filter for movies released between 2021 through 2024.
filtered_titles = [
    title for title in data.get('titles', [])
    if 2020 <= title.get('year', 0) <= 2024
][:3]

# Grab details for each filtered movie.
details_url_template = 'https://api.watchmode.com/v1/title/{}/details/'
results = []

for title in filtered_titles:
    title_id = title['id']
    title_name = title['title']
    release_year = title.get('year')

    details_url = details_url_template.format(title_id)
    detail_params = {'apiKey': api_key}
    detail_response = requests.get(details_url, params=detail_params)
    detail_data = detail_response.json()

    result = {
        'title_id': title_id,
        'title': title_name,
        'year': release_year,
        'user_rating': detail_data.get('user_rating'),
        'critic_rating': detail_data.get('critic_score'),
        'plot_overview': detail_data.get('plot_overview'),
    }
    results.append(result)

# Build the DataFrame
df = pd.DataFrame(results, columns=['title_id','title','year','user_rating','critic_rating','plot_overview'])

# Print the results.
for item in results:
    print(f"Title ID: {item['title_id']}")
    print(f"Title: {item['title']}")
    print(f"Year: {item['year']}")
    print(f"User Rating: {item['user_rating']}")
    print(f"Critic Rating: {item['critic_rating']}")
    print(f"Plot Overview: {item['plot_overview']}\n")

Title ID: 1498670
Title: Nosferatu
Year: 2024
User Rating: 7
Critic Rating: 81
Plot Overview: A gothic tale of obsession between a haunted young woman and the terrifying vampire infatuated with her, causing untold horror in its wake.

Title ID: 1674882
Title: The Substance
Year: 2024
User Rating: 7.2
Critic Rating: 84
Plot Overview: A fading celebrity decides to use a black market drug, a cell-replicating substance that temporarily creates a younger, better version of herself.

Title ID: 1678919
Title: Alien: Romulus
Year: 2024
User Rating: 7.3
Critic Rating: 72
Plot Overview: While scavenging the deep ends of a derelict space station, a group of young space colonizers come face to face with the most terrifying life form in the universe.



In [32]:
print(df)

   title_id           title  year  user_rating  critic_rating  \
0   1498670       Nosferatu  2024          7.0             81   
1   1674882   The Substance  2024          7.2             84   
2   1678919  Alien: Romulus  2024          7.3             72   

                                       plot_overview  
0  A gothic tale of obsession between a haunted y...  
1  A fading celebrity decides to use a black mark...  
2  While scavenging the deep ends of a derelict s...  
